# Day 3: Supply II - Enhancing the model

We will continue our modeling exercise by adding carbon taxes, renewable subsidies, and investment to the model.

This will allow us to consider the policy impacts of alternative energy transition policies.

The data and code are based on the paper "The Efficiency and Sectoral Distributional Implications of Large-Scale Renewable Policies," by Mar Reguant.

We first load relevant libraries, same as last session.

In [ ]:
#using Pkg
#Pkg.add(["DataFrames", "CSV", "JuMP", "Ipopt", "Plots", "Printf"])


In [ ]:
using DataFrames
using CSV
using JuMP
using Ipopt
using Plots
using Printf

Remember to set your path correctly:

## Building the model

We load the same data as last week, and also clean it up to simplify it further and create the demand and import curves.

In [ ]:
dfclust = CSV.read("data_jaere_clustered.csv", DataFrame);

# Re-scaling (we multiply by 8.76 to make it into a full year of hours (divided by 1000))
dfclust.weights = 8.76 * dfclust.weights / sum(dfclust.weights);

# Here only one demand type to make it easier
dfclust.demand = dfclust.q_residential + dfclust.q_commercial + dfclust.q_industrial;

# Calibrate demand based on elasticities (using 0.1 here as only one final demand)
elas = [.1, .2, .5, .3];
dfclust.b = elas[1] * dfclust.demand ./ dfclust.price;  # slope
dfclust.a = dfclust.demand + dfclust.b .* dfclust.price;  # intercept

# Calibrate imports (using elas 0.3)
dfclust.bm = elas[4] * dfclust.imports ./ dfclust.price;  # slope
dfclust.am = dfclust.imports - dfclust.bm .* dfclust.price;  # intercept

In [ ]:
dfclust

The technology file now includes the fixed cost of building new power plants (technologies 3-5). Note that we added an additional row for new natural gas plants.

We will use an annualization factor to pro-rate the importance of fixed costs for one year.

In [ ]:
tech = CSV.read("data_technology.csv", DataFrame);
afactor = (1 - (1 / (1.05^20.0))) / 0.05;
tech.F = tech.F ./afactor;
tech.F2 = tech.F2 ./afactor;
tech



### Adding investment

We are now ready to clear the market. We will **maximize welfare** using a non-linear solver.

$$ \max \ CS - Costs - Fixed Costs \\

\text{s.t.} \ \text{operational constraints, market clearing}. $$

Notice that we added the fixed costs to the problem, as we will be solving for the optimal level of wind and solar investment.

In [ ]:
## Clear market based on cost minimization
function clear_market_invest(data::DataFrame, tech::DataFrame)

    # We declare a model
    model = Model(
        optimizer_with_attributes(
            Ipopt.Optimizer, 
                "print_level"=>0)
        );

    # Set useful indexes
    I = nrow(tech);  # number of techs
    T = nrow(data);  # number of periods

    
    # Variables to solve for
    @variable(model, price[1:T]);
    @variable(model, demand[1:T]);
    @variable(model, imports[1:T]);
    @variable(model, quantity[1:T, 1:I] >= 0);
    @variable(model, costs[1:T]);
    @variable(model, gross_surplus[1:T]);
    @variable(model, gas_gw >= 0.0);
    @variable(model, wind_gw >= 0.0);
    @variable(model, solar_gw >= 0.0);
    @variable(model, profit_gas);
    @variable(model, profit_wind);
    @variable(model, profit_solar);

    
    # Maximize welfare including imports costs
    @NLobjective(model, Max, sum(data.weights[t] * 
                        (gross_surplus[t] - costs[t] ) for t=1:T)
                    - tech.F[5]*gas_gw  - tech.F[6]*wind_gw - tech.F[7]*solar_gw );

    # Market clearing
    @constraint(model, [t=1:T], 
        demand[t] == data.a[t] - data.b[t] * price[t]);
    @constraint(model, [t=1:T], 
        imports[t] == data.am[t] + data.bm[t] * price[t]);
    @constraint(model, [t=1:T], 
        demand[t] == sum(quantity[t,i] for i=1:I) + imports[t]);

    # Define surplus
    @constraint(model, [t=1:T], gross_surplus[t]==
                (data.a[t] - demand[t]) * demand[t] / data.b[t] 
            + demand[t]^2/(2*data.b[t]));

    # Define cost
    @constraint(model, [t=1:T], costs[t] ==
                    sum(tech.c[i] * quantity[t,i]
                    + tech.c2[i] * quantity[t,i]^2/2 for i=1:I)
        + (imports[t] - data.am[t])^2/(2 * data.bm[t]))

    # Constraints on output
    @constraint(model, [t=1:T], 
        quantity[t,1] <= data.hydronuc[t]);
    @constraint(model, [t=1:T,i=2:4], 
        quantity[t,i] <= tech[i,"capUB"]);
    @constraint(model, [t=1:T], 
        quantity[t,5] <= gas_gw);
    @constraint(model, [t=1:T], 
        quantity[t,6] <= wind_gw * data.wind_cap[t]);
    @constraint(model, [t=1:T], 
        quantity[t,7] <= solar_gw * data.solar_cap[t]);

    #Definition of profit
        @constraint(model, profit_gas == 
                            sum(data.weights[t]*(price[t] - tech.c[5] - tech.c2[5] * quantity[t,5])*quantity[t,5] for t=1:T) - tech.F[5]*gas_gw );
        @constraint(model, profit_wind == 
                            sum(data.weights[t]*price[t]*quantity[t,6] for t=1:T) - tech.F[6]*wind_gw );
        @constraint(model, profit_solar == 
                            sum(data.weights[t]*price[t]*quantity[t,7]  for t=1:T) - tech.F[7]*solar_gw );
    
    # Solve model
    optimize!(model);

    status = @sprintf("%s", JuMP.termination_status(model));

    if (status=="LOCALLY_SOLVED")
        p = JuMP.value.(price);
        avg_price = sum(p[t] * data.weights[t]/sum(data.weights) for t=1:T);
        q = JuMP.value.(quantity);
        imp = JuMP.value.(imports);
        d = JuMP.value.(demand);
        cost = JuMP.value.(costs);
        results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)),
            "avg_price" => avg_price,
            "price" => p,
            "quantity" => q,
            "imports" => imp,
            "demand" => d,
            "cost" => cost,
            "gas_gw" => JuMP.value.(gas_gw),
            "wind_gw" => JuMP.value.(wind_gw),
            "solar_gw" => JuMP.value.(solar_gw),
            "profit_gas" => JuMP.value.(profit_gas),
            "profit_wind" => JuMP.value.(profit_wind),
            "profit_solar" => JuMP.value.(profit_solar));
        return results
    else
        results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)));
        return results
    end

end

In [ ]:
results_min = clear_market_invest(dfclust, tech)

## Environmental regulation

We will begin by adding carbon taxes to the model.



In [ ]:
function clear_market_invest_tax(data::DataFrame, tech::DataFrame,tax=30.0)

    # We declare a model
    model = Model(
        optimizer_with_attributes(
            Ipopt.Optimizer, 
                "print_level"=>0)
        );

    # Set useful indexes
    I = nrow(tech);  # number of techs
    T = nrow(data);  # number of periods

    # Variables to solve for
    @variable(model, price[1:T]);
    @variable(model, demand[1:T]);
    @variable(model, imports[1:T]);
    @variable(model, quantity[1:T, 1:I] >= 0);
    @variable(model, costs[1:T]);
    @variable(model, gross_surplus[1:T]);
    @variable(model, gas_gw >= 0.0);
    @variable(model, wind_gw >= 0.0);
    @variable(model, solar_gw >= 0.0);

    # New variable
    @variable(model, totale[1:T]);

    # Maximize welfare including imports costs
    @NLobjective(model, Max, sum(data.weights[t] * 
                (gross_surplus[t] - costs[t] - tax*totale[t]) for t=1:T)
             - tech.F[5]*gas_gw  - tech.F[6]*wind_gw - tech.F[7]*solar_gw );

    # Market clearing
    @constraint(model, [t=1:T], 
        demand[t] == data.a[t] - data.b[t] * price[t]);
    @constraint(model, [t=1:T], 
        imports[t] == data.am[t] + data.bm[t] * price[t]);
    @constraint(model, [t=1:T], 
        demand[t] == sum(quantity[t,i] for i=1:I) + imports[t]);

    # Define surplus
    @constraint(model, [t=1:T], gross_surplus[t]==
                (data.a[t] - demand[t]) * demand[t] / data.b[t] 
            + demand[t]^2/(2*data.b[t]));

    # Define cost
    @constraint(model, [t=1:T], costs[t] ==
                    sum(tech.c[i] * quantity[t,i]
                    + tech.c2[i] * quantity[t,i]^2/2 for i=1:I)
        + (imports[t] - data.am[t])^2/(2 * data.bm[t]))

    # Define emissions 
    @constraint(model, [t=1:T], totale[t] ==
                    sum(tech.e[i] * quantity[t,i] + tech.e2[i] * quantity[t,i]^2 for i=1:I))

        # Constraints on output
    @constraint(model, [t=1:T], 
        quantity[t,1] <= data.hydronuc[t]);
    @constraint(model, [t=1:T,i=2:4], 
        quantity[t,i] <= tech[i,"capUB"]);
    @constraint(model, [t=1:T], 
        quantity[t,5] <= gas_gw);
    @constraint(model, [t=1:T], 
        quantity[t,6] <= wind_gw * data.wind_cap[t]);
    @constraint(model, [t=1:T], 
        quantity[t,7] <= solar_gw * data.solar_cap[t]);

    
    # Solve model
    optimize!(model);

    status = @sprintf("%s", JuMP.termination_status(model));

    if (status=="LOCALLY_SOLVED")
        p = JuMP.value.(price);
        avg_price = sum(p[t] * data.weights[t]/sum(data.weights) for t=1:T);
        q = JuMP.value.(quantity);
        imp = JuMP.value.(imports);
        d = JuMP.value.(demand);
        cost = JuMP.value.(costs);
        totale = JuMP.value.(totale);
        avg_emissions = sum(totale[t] * data.weights[t]/sum(data.weights) for t=1:T);
        results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)),
            "avg_price" => avg_price,
            "avg_emissions" => avg_emissions,
            "price" => p,
            "quantity" => q,
            "imports" => imp,
            "demand" => d,
            "cost" => cost,
            "totale" => totale,
            "gas_gw" => JuMP.value.(gas_gw),
            "wind_gw" => JuMP.value.(wind_gw),
            "solar_gw" => JuMP.value.(solar_gw));
        return results
    else
        results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)));
        return results
    end

end

In [ ]:
results_tax = clear_market_invest_tax(dfclust, tech)

Finally, we will add a subsidy to the model. One can think of the subsidy as a negative cost to renewable power. 

In [ ]:
function clear_market_invest_subsidy(data::DataFrame, tech::DataFrame; subsidy=20.0, renewable_charge =0.0)

    # We declare a model
    model = Model(
        optimizer_with_attributes(
            Ipopt.Optimizer, 
                "print_level"=>0)
        );

    # Set useful indexes
    I = nrow(tech);  # number of techs
    T = nrow(data);  # number of periods

    # Variables to solve for
    @variable(model, price[1:T]);
    @variable(model, demand[1:T]);
    @variable(model, imports[1:T]);
    @variable(model, quantity[1:T, 1:I] >= 0);
    @variable(model, costs[1:T]);
    @variable(model, gross_surplus[1:T]);
    @variable(model, gas_gw >= 0.0);
    @variable(model, wind_gw >= 0.0);
    @variable(model, solar_gw >= 0.0);

    # New variable
    @variable(model, totale[1:T]);

    # Maximize welfare including imports costs
    @NLobjective(model, Max, sum(data.weights[t] * 
                (gross_surplus[t] - costs[t] ) for t=1:T)
             - tech.F[5]*gas_gw  - tech.F[6]*wind_gw - tech.F[7]*solar_gw );

    # Market clearing
    @constraint(model, [t=1:T], 
        demand[t] == data.a[t] - data.b[t] * (price[t] + renewable_charge));
    @constraint(model, [t=1:T], 
        imports[t] == data.am[t] + data.bm[t] * price[t]);
    @constraint(model, [t=1:T], 
        demand[t] == sum(quantity[t,i] for i=1:I) + imports[t]);

    # Define surplus
    @constraint(model, [t=1:T], gross_surplus[t]==
                (data.a[t] - demand[t]) * demand[t] / data.b[t] 
            + demand[t]^2/(2*data.b[t]));

    # Define cost
    @constraint(model, [t=1:T], costs[t] ==
                    sum(tech.c[i] * quantity[t,i]
                    + tech.c2[i] * quantity[t,i]^2/2 
                    - subsidy * tech.renewable[i] * quantity[t,i] for i=1:I)
        + (imports[t] - data.am[t])^2/(2 * data.bm[t]))

    # Define emissions 
    @constraint(model, [t=1:T], totale[t] ==
        sum(tech.e[i] * quantity[t,i] + tech.e2[i] * quantity[t,i]^2 for i=1:I))

      
        # Constraints on output
    @constraint(model, [t=1:T], 
        quantity[t,1] <= data.hydronuc[t]);
    @constraint(model, [t=1:T,i=2:4], 
        quantity[t,i] <= tech[i,"capUB"]);
    @constraint(model, [t=1:T], 
        quantity[t,5] <= gas_gw);
    @constraint(model, [t=1:T], 
        quantity[t,6] <= wind_gw * data.wind_cap[t]);
    @constraint(model, [t=1:T], 
        quantity[t,7] <= solar_gw * data.solar_cap[t]);


    # Solve model
    optimize!(model);

    status = @sprintf("%s", JuMP.termination_status(model));

    if (status=="LOCALLY_SOLVED")
        p = JuMP.value.(price);
        avg_price = sum(p[t] * data.weights[t]/sum(data.weights) for t=1:T);
        q = JuMP.value.(quantity);
        imp = JuMP.value.(imports);
        d = JuMP.value.(demand);
        cost = JuMP.value.(costs);
        totale = JuMP.value.(totale);
        avg_emissions = sum(totale[t] * data.weights[t]/sum(data.weights) for t=1:T);
        subsidy_cost = sum(data.weights[t] * sum(subsidy * q[t,i] for i=6:7) for t=1:T);
        results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)),
            "avg_price" => avg_price,
            "avg_emissions" => avg_emissions,
            "price" => p,
            "quantity" => q,
            "imports" => imp,
            "demand" => d,
            "cost" => cost,
            "totale" => totale,
            "gas_gw" => JuMP.value.(gas_gw),
            "wind_gw" => JuMP.value.(wind_gw),
            "solar_gw" => JuMP.value.(solar_gw),
            "subsidy_cost" => subsidy_cost,
            "needed_charge" => (subsidy_cost/sum(data.weights[t] * d[t] for t=1:T)));
        return results
    else
        results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)));
        return results
    end

end

In [ ]:
results_subsidy = clear_market_invest_subsidy(dfclust, tech)

## Computing the renewable charge

We would like to add a constraint that states that the subsidies given to firms (solar and wind) need to equal the payments made by consumers with the renewable charges:
```
    # Subsidy charge 
    @constraint(model, 
        sum(data.weights[t] * sum(subsidy * quantity[t,i] for i=6:7) for t=1:T) == renewable_charge * sum(data.weights[t] * demand[t] for t=1:T));
```

One computational issue is that this is what is called a non-linear equation (`demand` and `renewable_charge` multiply each other, making it harder to compute).

It is best to proceed with a search approach for the renewable charge. We will code it with a simple loop here (akin to the visual search we saw last week).

We get intuition first without making it a function.

In [ ]:
current_diff = 1.0;
    guess = 5.0;
    while (current_diff > 1e-2)
        res = clear_market_invest_subsidy(dfclust, tech, renewable_charge=guess, subsidy=20.0);
        newguess = res["needed_charge"];
        current_diff = (guess-newguess).^2;
        guess = newguess;
    end
    

In [ ]:
guess

The result is telling us that the renewable charge should be about $8.643 per MWh consumed, we have found an equilibrium.

In [ ]:
res = clear_market_invest_subsidy(dfclust, tech, renewable_charge=8.965, subsidy=20.0)

### Making it into a function

We create a function that will do the loop and return the optimal solution.

In [ ]:
function clear_market_equilibrium(data::DataFrame, tech::DataFrame; 
     subsidy=0.0, renewable_charge=0.0)

    current_diff = 1.0;
    guess = subsidy/2.0;
    while (current_diff > 1e-2)
        res = clear_market_invest_subsidy(data, tech,  subsidy=subsidy, renewable_charge= guess);
        newguess = res["needed_charge"];
        current_diff = (guess-newguess).^2;
        guess = newguess
    end

    # we solve at the equilibrium to return results
    res = clear_market_invest_subsidy(data, tech,
             subsidy=subsidy, renewable_charge=guess);

    return res

end

In [ ]:
res_eq = clear_market_equilibrium(dfclust,tech, subsidy=10.0,renewable_charge=10.0)

## Follow-up exercises

1. Consider a tax and a subsidy that reach the same target of emissions. What are the different costs? How are the different components of welfare affected?

Note: This will require you to include emissions as an input or an output to the function.